In [1]:
# Step 1: Set up aws configure

In [2]:
# Step 2: Set up mlflow tracking server
import mlflow

mlflow.set_tracking_uri("http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/")

In [3]:
mlflow.set_experiment("Exp2: Bow vs Tfidf")

2025/04/09 14:47:15 INFO mlflow.tracking.fluent: Experiment with name 'Exp2: Bow vs Tfidf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://yt-plugin-bucket/980134970665471660', creation_time=1744190234933, experiment_id='980134970665471660', last_update_time=1744190234933, lifecycle_stage='active', name='Exp2: Bow vs Tfidf', tags={}>

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import matplotlib.pyplot as plt 
import seaborn as sns

In [6]:
df = pd.read_csv('data/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [7]:
df.shape

(36662, 2)

In [8]:

# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


2025/04/09 14:48:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660/runs/ce73a391adbc423f9cca6c41fdd4fad7
🧪 View experiment at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660


2025/04/09 14:49:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660/runs/1262096555994a2f9bd2aeee656bf95f
🧪 View experiment at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660


2025/04/09 14:50:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660/runs/479a095057b74fbabd16efbf3fbd0798
🧪 View experiment at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660


2025/04/09 14:52:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660/runs/f50aed2c6df144b9afe248a0620fb6a0
🧪 View experiment at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660


2025/04/09 14:55:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660/runs/2ab0fa7d9e8c4f2b921ca748748f2ec5
🧪 View experiment at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/980134970665471660
